In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import numpy as np
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
## Read the pdfs from folder

loader = PyPDFDirectoryLoader("./us_census")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap =200)

final_docs = text_splitter.split_documents(documents)

final_docs[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, 

In [3]:
len(final_docs)

316

In [4]:
embedding = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

C:\Users\biswa\AppData\Local\Temp\ipykernel_5136\386046288.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceBgeEmbeddings(
c:\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(np.array(embedding.embed_query(final_docs[0].page_content)))
print(np.array(embedding.embed_query(final_docs[0].page_content)).shape)

[-5.21856621e-02  8.78243148e-02 -2.12525427e-02 -9.72598940e-02
  5.37132891e-03  2.62250081e-02  3.26021463e-02  4.01220582e-02
  5.75089641e-02 -1.99616887e-02  6.69143423e-02  3.79592963e-02
  1.96072236e-02  4.70315255e-02  4.54055592e-02 -3.78638543e-02
  3.18033225e-03  6.45747930e-02 -3.91166052e-03 -2.14959262e-03
 -5.02138920e-02  9.49937943e-03 -4.15326282e-02  4.91290390e-02
  3.67074907e-02  7.36322440e-03  6.87897846e-04 -1.88188031e-02
 -3.10556740e-02 -5.20810187e-02  7.69521594e-02 -1.87770016e-02
 -2.20095646e-02 -1.00744918e-01  2.17985394e-06 -3.10376845e-02
 -1.24892332e-02  4.74395677e-02  1.10889915e-02 -4.53370847e-02
 -1.93461757e-02 -7.08796382e-02 -5.53427003e-02  2.04936359e-02
  1.56286452e-02 -4.54037599e-02 -1.03025939e-02  3.83944437e-02
 -3.65664028e-02 -5.69726247e-03 -1.79586951e-02 -2.71443021e-03
 -4.97777089e-02 -1.29989656e-02 -1.30686173e-02 -8.66997242e-03
  1.17883021e-02 -2.00637542e-02  5.09997718e-02 -9.59499553e-02
  8.35938100e-03  2.84801

In [6]:
vectorDB=FAISS.from_documents(final_docs,embedding)

In [7]:
## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorDB.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [8]:
retriever = vectorDB.as_retriever(search_type = "similarity", search_kwargs = {"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002100AFF5BA0> search_kwargs={'k': 3}


In [9]:
HUGGINGFACEHUB_API_TOKEN = os.environ['HUGGINGFACEHUB_API_TOKEN']

In [26]:
from langchain_community.llms import HuggingFaceHub
# from langchain_community.llms import HuggingFaceEndpoint

# hf = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta")
hf = HuggingFaceHub(
    repo_id="openai-community/gpt2",
    model_kwargs = {"temperature":0.1,"max_length":500},
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN
)

query="What is the health insurance coverage?"
hf.invoke(query)

AttributeError: 'InferenceClient' object has no attribute 'post'

In [14]:
# from langchain_huggingface import HuggingFaceEndpoint



In [ ]:
# #Hugging Face models can be run locally through the HuggingFacePipeline class.
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id="meta-llama/Llama-3.1-8B-Instruct",
#     task="text-generation",
#     pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
# )

# llm = hf 
# llm.invoke(query)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-68664713-3f11a7a7469b8ba11511b494;13eef1c7-48f2-4702-a1e3-cb9c997d63b7)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [15]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}

Question:{question}

Helpful Answers:
 """

In [16]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [17]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt":prompt}
)

In [18]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [19]:
# Calling the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

AttributeError: 'InferenceClient' object has no attribute 'post'